# Optimization

## Import Libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd

from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.core.problem import ElementwiseProblem

import src.optimization_utils as ou
import src.solutions as s


## Load Model

In [2]:
bundle = ou.load_model_bundle("../artifacts/mech_fail_model.joblib")

In [3]:
FEATURES = bundle["features"]
print(f"Training features ({len(FEATURES)}): {FEATURES}")


Training features (24): ['age', 'sex', 'PI_preop', 'PT_preop', 'LL_preop', 'SS_preop', 'T4PA_preop', 'L1PA_preop', 'SVA_preop', 'cobb_main_curve_preop', 'FC_preop', 'tscore_femneck_preop', 'HU_UIV_preop', 'HU_UIVplus1_preop', 'HU_UIVplus2_preop', 'num_levels', 'UIV_implant', 'num_fused_levels', 'ALIF', 'XLIF', 'TLIF', 'num_rods', 'num_screws', 'osteotomy']


In [4]:
UIV_CHOICES, xl, xu = ou.get_decision_config()

In [5]:
print("UIV_CHOICES:", UIV_CHOICES)
print("xl:", xl)
print("xu:", xu)

cols = ["uiv_code", "num_levels", "ALIF", "XLIF", "TLIF", "num_rods", "num_pelvic_screws", "osteotomy"]
print(pd.DataFrame([xl, xu], index=["xl","xu"], columns=cols))

UIV_CHOICES: ['Hook', 'PS', 'FS']
xl: [0 0 0 0 0 1 1 0]
xu: [2 5 1 1 1 6 6 1]
    uiv_code  num_levels  ALIF  XLIF  TLIF  num_rods  num_pelvic_screws  \
xl         0           0     0     0     0         1                  1   
xu         2           5     1     1     1         6                  6   

    osteotomy  
xl          0  
xu          1  


## Test Patient w fixed parameters

In [6]:
patient_fixed = {
    "age": 65,
    "sex": "FEMALE",
    "bmi": 18.48,
    "C7CSVL_preop": -6.1,
    "SVA_preop": 40.7,
    "T4PA_preop": 7.4,
    "L1PA_preop": 9.1,
    "LL_preop": 51.8,
    "L4S1_preop": 32.5,
    "PT_preop": 14.1,
    "PI_preop": 45.8,
    "SS_preop": 31.7,
    "cobb_main_curve_preop": 60.2,
    "FC_preop": 10.6,
    "tscore_femneck_preop": -0.5,
    "HU_UIV_preop": 229,
    "HU_UIVplus1_preop": 245,
    "HU_UIVplus2_preop": 248,
}

ou.debug_candidate(
    x=np.array([0, 3, 1, 0, 0, 6, 4, 0]),
    patient_fixed=patient_fixed,
    bundle=bundle,
    uiv_choices=UIV_CHOICES,
)

{'x': [0, 3, 1, 0, 0, 6, 4, 0],
 'plan': {'UIV_implant': 'Hook',
  'num_interbody_fusion_levels': 3,
  'ALIF': 1,
  'XLIF': 0,
  'TLIF': 0,
  'num_rods': 6,
  'num_pelvic_screws': 4,
  'osteotomy': 0},
 'mech_fail_prob': 0.05953365223258485,
 'fitness': 0.05953365223258485}

## Build optimization problem

In [7]:
def make_problem():
    def _evaluate(self, x, out, *args, **kwargs):
        f = ou.fitness_mech_fail_only(x, patient_fixed, bundle, UIV_CHOICES)
        out["F"] = np.array([f], dtype=float)

    ProblemType = type(
        "SpineProblem",
        (ElementwiseProblem,),
        {
            "__init__": lambda self: ElementwiseProblem.__init__(
                self,
                n_var=len(xl),
                n_obj=1,
                xl=xl,
                xu=xu,
                vtype=int,
            ),
            "_evaluate": _evaluate,
        },
    )
    return ProblemType()

problem = make_problem()

## Run GA and view results

In [8]:
algorithm = GA(
    pop_size=100,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=3),
    eliminate_duplicates=True,
)

res = minimize(
    problem,
    algorithm,
    get_termination("n_gen", 15),
    seed=42,
    verbose=True,
    save_history=True
)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       99 |  0.5606542895 |  0.0426623250
     2 |      199 |  0.0687243187 |  0.0405743649
     3 |      299 |  0.0544337896 |  0.0405743649
     4 |      399 |  0.0468680927 |  0.0405743649
     5 |      499 |  0.0441870896 |  0.0405743649
     6 |      599 |  0.0422238533 |  0.0405743649
     7 |      699 |  0.0418689001 |  0.0405743649
     8 |      799 |  0.0410545957 |  0.0405743649
     9 |      899 |  0.0405743649 |  0.0405743649
    10 |      999 |  0.0405743649 |  0.0405743649
    11 |     1099 |  0.0405743649 |  0.0405743649
    12 |     1199 |  0.0405743649 |  0.0405743649
    13 |     1299 |  0.0405743649 |  0.0405743649
    14 |     1399 |  0.0405743649 |  0.0405743649
    15 |     1499 |  0.0405743649 |  0.0405743649


## Show best solution

In [9]:
best_x = np.asarray(res.X).astype(int)
best_plan = ou.decode_plan(best_x, UIV_CHOICES)

best_full = ou.build_full_input(patient_fixed, best_plan)
best_prob = ou.predict_mech_fail_prob(best_full, bundle)

print("Best mech_fail_prob:", best_prob)
best_plan

Best mech_fail_prob: 0.040574364851296424


{'UIV_implant': 'PS',
 'num_interbody_fusion_levels': 0,
 'ALIF': 1,
 'XLIF': 1,
 'TLIF': 0,
 'num_rods': 6,
 'num_pelvic_screws': 1,
 'osteotomy': 0}

## Getting multiple solutions

Currently forcing ALIF to 1 for all solutions because that is all we can evaluate with training data

**Get top 10 solutions from last generation**

In [10]:
pop = res.algorithm.pop
X_pop = pop.get("X")
F_pop = pop.get("F").flatten()

order = np.argsort(F_pop)

TOP_SLICE = 1500
TOP_N = 10

rows = []
for idx in order[:TOP_SLICE]:
    x = np.asarray(X_pop[idx]).astype(int)
    plan = ou.decode_plan(x, UIV_CHOICES)
    rows.append({
        **plan,
        "fitness": float(F_pop[idx]),
    })

df = pd.DataFrame(rows)

df_unique = df.drop_duplicates(
    subset=["UIV_implant", "num_interbody_fusion_levels", "ALIF", "XLIF", "TLIF",
            "num_rods", "num_pelvic_screws", "osteotomy"],
    keep="first",
).reset_index(drop=True)

df_unique.head(TOP_N)

,UIV_implant,num_interbody_fusion_levels,ALIF,XLIF,TLIF,num_rods,num_pelvic_screws,osteotomy,fitness
0,PS,0,1,1,0,6,1,0,0.040574
1,PS,0,0,1,0,6,2,0,0.040574
2,PS,0,0,1,0,6,3,0,0.040574
3,PS,2,0,1,0,6,3,0,0.040574
4,PS,2,0,1,0,6,1,0,0.040574
5,PS,1,0,1,0,6,1,0,0.040574
6,PS,0,0,1,0,6,5,0,0.040574
7,PS,4,0,1,0,6,5,0,0.040574
8,PS,1,0,1,0,6,2,0,0.040574
9,PS,2,0,1,0,6,5,0,0.040574


In [11]:
df["UIV_implant"].value_counts()


UIV_implant
PS    100
Name: count, dtype: int64

In [12]:
solutions = s.get_diverse_solutions(
    res=res,
    top_n=5,
    top_per_gen=50,
    eps=0.1,
    bucket_cols=("UIV_implant", "XLIF", "TLIF"),
    n_per_bucket=1,
)

solutions


,UIV_implant,num_interbody_fusion_levels,ALIF,XLIF,TLIF,num_rods,num_pelvic_screws,osteotomy,fitness,gen
0,PS,0,1,1,0,6,1,0,0.040574,1
1,PS,4,0,0,0,6,5,0,0.042662,0
2,Hook,3,0,1,0,6,1,0,0.056669,1
3,Hook,0,0,0,0,5,3,0,0.065024,0
4,FS,0,0,0,0,5,4,0,0.077422,0
